In [51]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from makemore_helper import *

import matplotlib.pyplot as plt
%matplotlib inline
import random

In [52]:
with open('shakespere.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [53]:
len(text)

1115389

In [54]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
''.join(chars), vocab_size

("\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz", 65)

In [55]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for ch, i in stoi.items()}
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda s: ''.join([itos[ch] for ch in s])

print(encode('Hey there, this is Luv'))
print(decode(encode('Hey there, this is Luv')))

[20, 43, 63, 1, 58, 46, 43, 56, 43, 6, 1, 58, 46, 47, 57, 1, 47, 57, 1, 24, 59, 60]
Hey there, this is Luv


In [56]:
data = torch.tensor(encode(text))
data.shape, data.dtype, data[:1000]

(torch.Size([1115389]),
 torch.int64,
 tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
         53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
          1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
         57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
          6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
         58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
          1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
         53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
         57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
          8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
          1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
         53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39

In [57]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [58]:
block_size=8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [59]:
torch.manual_seed(1337)
batch_size = 4 # sequences to be processed in parellel
block_size = 8 # context legnth for predictions

def get_batch(split):
    data = train_data if split == 'train' else val_data
    # ix is tensor of random starts of batches
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
# input
print(f'{xb.shape=}')
print(f'{xb=}')
#output
print(f'{yb.shape=}')
print(f'{yb=}')

xb.shape=torch.Size([4, 8])
xb=tensor([[59, 57,  1, 58, 56, 39, 47, 58],
        [30, 10,  0, 15, 53, 51, 43,  6],
        [34, 21, 26, 15, 17, 26, 32, 21],
        [27, 10,  0, 32, 46, 53, 59,  1]])
yb.shape=torch.Size([4, 8])
yb=tensor([[57,  1, 58, 56, 39, 47, 58, 53],
        [10,  0, 15, 53, 51, 43,  6,  1],
        [21, 26, 15, 17, 26, 32, 21, 27],
        [10,  0, 32, 46, 53, 59,  1, 42]])


In [118]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are (B, T) tensors
        logits = self.token_embedding_table(idx) # B,T, C

        # we have B, T, C matrix but pytorch wants C as second dim 
        if targets is None:
           loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            #get predictions
            logits, __ = self(idx)
            # get last time step
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probs
            probs = F.softmax(logits, dim=1) # (B,C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx

m = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(logits)
print(loss)

torch.Size([256, 65])
tensor([[-0.1600,  1.3981, -0.7047,  ..., -1.9908,  0.8574, -2.1603],
        [-1.4731,  0.8040, -0.1425,  ...,  0.7180,  0.3684,  2.2792],
        [-0.2060,  1.5973,  0.1185,  ...,  0.5812, -0.5356, -1.7944],
        ...,
        [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
        [ 1.0726,  0.7295, -0.6665,  ...,  0.3115, -1.7675,  0.6818],
        [ 1.1513,  1.0539,  3.4105,  ..., -0.5686,  0.9079, -0.1701]],
       grad_fn=<ViewBackward0>)
tensor(4.6678, grad_fn=<NllLossBackward0>)


In [119]:
print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [120]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [121]:
# training params
batch_size = 32
steps = 100000
log_step = steps // 10

In [122]:
# training
for step in range(steps):

    # getting the batch
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = m(xb, yb)
    if step % log_step == 0:
        print(f'{step} / {steps} ==> {loss.item()}')
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

0 / 100000 ==> 4.776141166687012
10000 / 100000 ==> 2.600294828414917
20000 / 100000 ==> 2.5083565711975098
30000 / 100000 ==> 2.562260389328003
40000 / 100000 ==> 2.4361395835876465
50000 / 100000 ==> 2.5112979412078857
60000 / 100000 ==> 2.4667978286743164
70000 / 100000 ==> 2.4566853046417236
80000 / 100000 ==> 2.4076743125915527
90000 / 100000 ==> 2.4973397254943848
2.4009931087493896


In [124]:
print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Fomourmenghau buaror VOubed spo mng as chathab llll:
Ware,

ee
Thy,
Thooured aly y hindr's.
Fashat--
MNGess am hare hathure Anfaneof f s llon!

ICLiroushanot

Then
Magend cugss, be jollrty

AROUFLom, ifay wil whes, gheatalloult llats howheprshakengayoref f f abighine ck orors n s? bimyerd Foutheig vemy.
NG t isoststor hnor 'myougorme whe s'car n r toun t pridie awe he of t ad
BY:
Hatamethat vint ibe, ghyone'lisurerat ms The Bulin the theid IAm VOMal fis,
OMarethay:

MESheelore f bu hasen, t 'Wha


In [125]:
# trick to do weighted mean for self attention
torch.manual_seed(1337)

B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])